In [10]:
import os

from raman_lib.spectra_scoring import score_sort_spectra
from raman_lib.misc import load_data

In [11]:
name = "Biofilme"

path = "./data"

In [12]:
data = load_data(path)

W+1B
1B
2B
W
W+2B
PW


In [13]:
print(data.label.value_counts())

1B      48
W       32
2B      16
W+2B    16
W+1B    15
PW      15
Name: label, dtype: int64


In [14]:
data_new = score_sort_spectra(data, min_score=5000, min_n=10)

Analyzed 142 spectra in 2.92 seconds.

Mean Score: 22791

1st Quartile: 9622
Median Score: 14744
3rd Quartile: 26955

Min Score: 365
Max Score: 64676


In [15]:
print(data_new.label.value_counts())

1B      48
W       32
W+2B    16
W+1B    15
2B      10
PW      10
Name: label, dtype: int64


In [16]:
filename = name.lower().replace(" ", "_") + ".csv"

if not os.path.exists("./results_qc/"):
    os.mkdir("./results_qc/")

data_new.to_csv(os.path.join("./results_qc/", filename), index=False)